## Configuration

In [1]:
### Configuration
# Import dependencies
import os
from collections import Counter
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in current song candidates

In [3]:
# Get playlist IDs
playlistID1 = getPlaylistID(credentials, 'Summer - Candidate Songs')
# playlistID2 = getPlaylistID(credentials, '2023 Round 2')

# Read in CurrentYearSongCandidates playlist
playlistTracks1 = fetchTracksFromPlaylist(credentials, playlistID1)
#playlistTracks2 = fetchTracksFromPlaylist(credentials, playlistID2)

# Merge dfs
#allNewTracks = pd.concat([playlistTracks1, playlistTracks2], axis=0, ignore_index=True)
allNewTracks = playlistTracks1
allNewTracks

,artistName,artistID,songName,release_date,songURI,songLocation,albumLength,songDuration,songExplicit,songPopularity
0,Kari Faux,4c2ighP1wj8E5dVGJDCOiB,WHITE CAPRICE,2023-05-26,spotify:track:3Q0h7wkHLIlwySLGZYqFPu,8,13,196163,True,32
1,Gangsta Boo,3ppZNqihWOzuH4A0f4KmeP,WHITE CAPRICE,2023-05-26,spotify:track:3Q0h7wkHLIlwySLGZYqFPu,8,13,196163,True,32
2,Portraits Of Tracy,34QizBjxVeMcA5HlZpKxxE,Found!,2023-05-26,spotify:track:1FgY491AT8WYmMgZgWLfkz,16,18,305511,False,6
3,Kavinsky,0UF7XLthtbSF2Eur7559oV,Nightcall,2010,spotify:track:0U0ldCRmgCqhVvD6ksG63j,1,7,258413,False,73
4,easy life,7uwY65fDg3FVJ8MkJ5QuZK,OTT,2022-10-07,spotify:track:7vfVFDOOwErIDLt2vi2Q3W,6,15,157817,False,44
...,...,...,...,...,...,...,...,...,...,...
874,PinkPantheress,78rUTD7y6Cy67W1RVzYs7t,Mosquito,2023-11-10,spotify:track:3DrkYwyu8QqZQ05r2gGqHi,3,13,146240,False,66
875,PinkPantheress,78rUTD7y6Cy67W1RVzYs7t,The aisle,2023-11-10,spotify:track:6SjxVgMjH3b3AI4wVviZfi,4,13,165773,False,61
876,Balkan Paradise Orchestra,5tWhPlOVHAdgPbtzTR95Hs,Cheesy,2024-02-23,spotify:track:6xoUFb5z8prxFuigUhvcMC,2,11,169329,False,22
877,Balkan Paradise Orchestra,5tWhPlOVHAdgPbtzTR95Hs,Estimar,2024-02-23,spotify:track:4WZgVlwv6RJnOHTG7AADk0,5,11,173122,False,22


## Read in old playlists

In [4]:
# Read in all old playlists
oldPlaylists = ["Summer 2018", "Summer 2019", "Endless Summer 2020", "Endless Summer 2021", "Endless Summer 2022", "Endless Summer 2023"]
# Empty df to hold old songs
allOldTracks = pd.DataFrame()
# Loop through playlists
for playlist in oldPlaylists:
    # Get playlist ID
    playlist_givenYear = getPlaylistID(credentials, playlist)
    # Get tracks from that playlist
    playlistTracks_givenYear = fetchTracksFromPlaylist(credentials, playlist_givenYear)
    # Append to df
    allOldTracks = pd.concat([allOldTracks, playlistTracks_givenYear], axis=0, ignore_index=True)
allOldTracks
# pl2018 = getPlaylistID(credentials, "Summer 2018")
# plt2018 = fetchTracksFromPlaylist(credentials, pl2018)
# pl2019 = getPlaylistID(credentials, "Summer 2019")
# plt2019 = fetchTracksFromPlaylist(credentials, pl2019)
# pl2020 = getPlaylistID(credentials, "Endless Summer 2020")
# plt2020 = fetchTracksFromPlaylist(credentials, pl2020)
# pl2021 = getPlaylistID(credentials, "Endless Summer 2021")
# plt2021 = fetchTracksFromPlaylist(credentials, pl2021)
# pl2022 = getPlaylistID(credentials, "Endless Summer 2022")
# plt2022 = fetchTracksFromPlaylist(credentials, pl2022)
# # Merge dfs
# allOldTracks = pd.concat([plt2018, plt2019, plt2020, plt2021, plt2022], axis=0, ignore_index=True)

,artistName,artistID,songName,release_date,songURI,songLocation,albumLength,songDuration,songExplicit,songPopularity
0,Gryffin,2ZRQcIgzPCVaT9XKhXZIzh,Winnebago (feat. Quinn XCII & Daniel Wilson),2018-04-20,spotify:track:2FVm4ADgYRIQWlpYKDPdl9,1,1,179936,True,49
1,Quinn XCII,3ApUX1o6oSz321MMECyIYd,Winnebago (feat. Quinn XCII & Daniel Wilson),2018-04-20,spotify:track:2FVm4ADgYRIQWlpYKDPdl9,1,1,179936,True,49
2,Daniel Wilson,44eNlYXYkIGob3NU82tgLq,Winnebago (feat. Quinn XCII & Daniel Wilson),2018-04-20,spotify:track:2FVm4ADgYRIQWlpYKDPdl9,1,1,179936,True,49
3,Watsky,3mJ9GlkLzj8Ka7Z7EQaCMi,Sloppy Seconds,2013-01-01,spotify:track:0Vwfd6fxFrL3kCnZSJ9vid,11,17,245933,True,0
4,The National,2cCUtGK9sDU2EoElnk0GNB,The System Only Dreams in Total Darkness,2017-09-08,spotify:track:1HAzsv7VCvbgqrpenTRWj1,4,12,236746,False,0
...,...,...,...,...,...,...,...,...,...,...
1885,Dawes,0CDUUM6KNRvgBFYIbWxJwV,Who Do You Think You're Talking To?,2020-10-02,spotify:track:3JRKgmK00uNLQzXGcLUaZ3,6,9,199413,False,33
1886,Amerigo Gazaway,0Bvwcw5e2SPEifNEORZTZf,Hold Me Tight,2019-06-21,spotify:track:0iIdSq9HOjnnXoly9ayWWe,3,12,148250,False,8
1887,Billy Talent,08yf5A2nS4XEeNvabDXqyg,Reckless Paradise,2022-01-21,spotify:track:6sQD8a5zZ09yA60YkeqDRS,2,10,204076,False,0
1888,Yeasayer,04HvbIwBccFmRie5ATX4ft,Fluttering In The Floodlights,2019-04-10,spotify:track:1ZEEm3E9Yvho5mYyzGFvxy,1,1,198260,False,34


## Filter out non-qualifiers

In [5]:
# Filter out any songs that have already appeared in old playlists
# Based on songURI alone
newFiltered = allNewTracks.loc[~allNewTracks['songURI'].isin(allOldTracks['songURI'])]
print(len(newFiltered))
# Based on artist ID + trackName, which is more exact
newFiltered['combined'] = newFiltered['artistID']+newFiltered['songName']
allOldTracks['combined'] = allOldTracks['artistID']+allOldTracks['songName']
newFiltered = newFiltered.loc[~newFiltered['combined'].isin(allOldTracks['combined'])]
print(len(newFiltered))

# Filter out any songs older than 5 years
dateMin = datetime((datetime.today() - relativedelta(years=5)).year, 3, 17)
newFiltered['release_date'] = pd.to_datetime(newFiltered['release_date'])
newFiltered = newFiltered.loc[newFiltered['release_date']>dateMin].reset_index(drop=True)
print(len(newFiltered))

# Sort by frequency of artist, artist
# Get frequency of artists in previous summers
frequency = pd.DataFrame(sorted(Counter(allOldTracks['artistID']).items(), key=lambda x:x[1], reverse=True))
frequency.columns = ['artistID', 'artistAppearances']

# Merge with new 
newFiltered = newFiltered.merge(frequency, how='left', on='artistID')
newFiltered['artistAppearances'] = newFiltered['artistAppearances'].fillna(0)

# Sort
newFiltered = newFiltered.sort_values(by=['artistAppearances', 'artistID', 'release_date'], ascending=False, ignore_index=True)

# Drop duplicates
newFiltered = newFiltered.drop_duplicates(subset='songURI', ignore_index=True)
newFiltered = newFiltered.drop_duplicates(subset=['artistID', 'songName'], ignore_index=True)
print(len(newFiltered))

879
875
872
657


## Create new playlist

In [6]:
# Warning: This will make a new playlist
playlistNew = getPlaylistID(credentials, 'Summer - Candidate Songs 2')
# Add songs to the playlist
addToPlaylist(credentials, newFiltered['songURI'].to_list(), playlistNew)

# Create playlist buckets to hold songs that are being selected

In [3]:
# Warning: This will make a new playlist
playlist10 = getPlaylistID(credentials, 'Summer - Top 10 Candidates')
playlist100 = getPlaylistID(credentials, 'Summer - Top 100 Candidates')
playlist300 = getPlaylistID(credentials, 'Summer - Top 300 Candidates')
playlist500 = getPlaylistID(credentials, 'Summer - Top 500 Candidates')

TypeError: 'NoneType' object is not iterable